# PREDICCION DE FASE USANDO NPL

## 1. Preparacion

### 1.1 Importe De Librerias

In [1]:
import pandas as pd
import numpy as np
from numpy import loadtxt
from pylab import *
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell  
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display, HTML
from scipy import stats
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
from statistics import mean

import re
import nltk
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords
from sklearn import preprocessing
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn.model_selection import RandomizedSearchCV
import pickle
import nltk
from sklearn.metrics import f1_score
from sklearn.metrics import plot_confusion_matrix

from dash import Dash, dash_table
import dash_html_components as html
from dash import Dash, Input, Output, callback, dash_table
import pandas as pd
import dash_bootstrap_components as dbc

from datetime import datetime

[nltk_data] Downloading package stopwords to C:\Users\Diego
[nltk_data]     Ojeda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\DIEGOO~1\AppData\Local\Temp/ipykernel_14212/2342114825.py:36: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
path='Data.xlsx'
base = pd.read_excel(path)
base.Nombre.unique()

array(['MOR BL 04', 'MOR BL 05', 'MOR BL 07', 'MOR BL 08', 'MOR AL 04',
       'MOR AL 05', 'MOR AL 01', 'MOR AL 07', 'MOR AL 03',
       'MORICHE AO-09', 'MORICHE N-08', 'MORICHE N 03', 'MORICHE N-11',
       'MORICHE N 02', 'MORICHE NORTE 08', 'MORICHE NORTE 19',
       'MORICHE NORTE 17', 'MORICHE O 11', 'MORICHE O 10', 'MORICHE O 04',
       'MORICHE O 08', 'MORICHE O 05', 'MORICHE O 09', 'MORICHE AN2 01',
       'MORICHE AN2-03', 'MOR AO 06', 'MOR AO 10', 'MORICHE CC 02',
       'MORICHE CC 01', 'MORICHE CC 04', 'MORICHE CC 09', 'MORICHE CC 03',
       'MORICHE CC 05', 'MORICHE CC 10', 'MOR U 02 ', 'MOR U 05',
       'MOR U 07', 'MOR B 03', 'MOR B 05', 'MOR B 01', 'MOR AI 08',
       'MOR AV 05', 'MOR A DESVIADO 2', 'MOR AI 04', 'MOR AI 07',
       'MOR AU 18', 'MOR AU 12', 'MOR AU 02 ', 'MOR AU 17', 'MOR AU 16',
       'MORICHE BA 10', 'MORICHE BA 15', 'MORICHE BA 02', 'MORICHE BA 13',
       'MORICHE BD 06', 'MORICHE BD 02', 'MOR BR 02', 'MOR BR 04 ',
       'MOR BR 08 ', 'MOR A

In [3]:
def pre_pro_well(base,well):
    '''
    FUNCION QUE SELECCIONA UN POZO, TRANSFORMA LOS DATOS COMO INPUT DEL MODELO

    - base: Dataframe con los datos de todos los pozos.
    - well: cadena de caracteres con el nombre del pozo a evaluar    
    '''

    # Seleccion del pozo a preprocesar
    base = base[base["Nombre"]==well]
    
    #return base

    # Seleccion de la columna de tesxto
    X = base["Operación"]
    X = X.reset_index(drop=True)

    # Libreria de stopwords en ingles
    nltk.download('wordnet')

    # Lista de textos vacia
    documents = []
    
    # Instanciacion del lematizador en ingles
    stemmer = WordNetLemmatizer()
    
    #return X

    # Ciclo for para que recorra todos las filas con las actividaes (textos)
    for sen in range(0, len(X)):

        # Removemos los caracteres especiales
        document = re.sub(r'\W', ' ', str(X[sen]))
        
        # Removemos todos las cadenas de caracteres de un solo caracter
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
        
        # Removemos caracteres sencillso al inicio
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
        
        # Substituimos dobles espacios por espacios sencillos
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removemos caracteres numericos
        document = re.sub(r'\[[0-9]*\]',' ',document)
        
        # Convertimos todo a minusculas
        document = document.lower()
        
        # Realizamos Lematizacion
        document = document.split()

        document = [stemmer.lemmatize(word) for word in document]
        document = ' '.join(document)
        
        documents.append(document)
    
    return (documents,base)

def identificacion_OCM(well):
    '''
    Función que usa un modelo de NPL para identificar las actividades y encontrar el inicio de OCM
    ----------------------------------------------------------------------------------------------

    - well = Cadena de caracteres con el nombre del pozo a probar

    Retorna
    - 
    '''
    #-----------------------------------------------------------------------------
    # LECTURA Y PREPARACION DE DATOS
    #-----------------------------------------------------------------------------

    # Lectura de Datos
    path='Data.xlsx'
    base = pd.read_excel(path)
    base = base.dropna()
    base = base.reset_index(drop=True)

    data = base

    # Eliminacion de columnas innecesarias y cambio de nombre de columnas
    data = data.rename({'Operación':"Operacion"}, axis=1)

    #-----------------------------------------------------------------------------
    # CARGA DE MODELOS
    #-----------------------------------------------------------------------------

    # Cargue del modelo entrenado
    model_rf = pickle.load(open("rf.sav", 'rb'))

    # Cargue del vectorizador entrenado
    vectorizer = pickle.load(open("vectorizer.sav", 'rb'))

    # Cargue del codificador de etiquetas
    le = pickle.load(open("label_encoder.sav", 'rb'))

    #-----------------------------------------------------------------------------
    # PREPROCESAMIENTO DE LA INFORMACION PARA LA PREDICCION DEL POZO SOLICITADO
    #-----------------------------------------------------------------------------

    # Preparamos los datos para la prediccion sobre un pozo especifico
    X_val,base_val = pre_pro_well(base,well)
    #return pre_pro_well(base,well)

    # Usamos el vectorizador para transformar los datos para la prediccion
    X_val = vectorizer.transform(X_val)

    #-----------------------------------------------------------------------------
    # PREDICCION SOBRE DATOS PREPROCESADOS
    #-----------------------------------------------------------------------------

    # Prediccion
    y_val = model_rf.predict(X_val)

    #-----------------------------------------------------------------------------
    # VISUALIZACION DE LA PREDICCION
    #-----------------------------------------------------------------------------

    # Union con datos del pozo
    base_val["Target"] = le.inverse_transform(y_val)

    # Organizacion del dataframe
    base_val = base_val[['Siglas', 'Target', 'Nombre', 'Desde', 'Hasta',"MD From (ft)","MD to (ft)","Codigo","Subcodigo","Operación"]]
    
    #-----------------------------------------------------------------------------
    # BUSQUEDA DE INICIO DE OCM
    #-----------------------------------------------------------------------------

    # Organizacion de la base en orden descendente
    base_val = base_val.sort_values(by=['Desde'],ascending=False)
    base_val = base_val.reset_index(drop=True)

    # Algoritmo para deteccion de inicio de OCM
    fila = 1
    while (base_val.Target[fila] == base_val.Target[fila-1]):
        fila=fila+1
    else:
        ID_OCM=fila

    ODR_Finish = base_val.Hasta[ID_OCM]

    base_val = base_val.sort_values(by=['Desde'],ascending=True)
    base_val = base_val.reset_index(drop=True)

    ODR_Finish = ODR_Finish.strftime("%m/%d/%Y %H:%M")
    
    respuesta =str("El Final del evento ODR en el Pozo " + str(well) + " se da en la fecha: " + str(ODR_Finish)+"\n")
    
    return(base_val,respuesta,ODR_Finish)

In [4]:
df,respuesta, ODR_Finish = identificacion_OCM("MOR BL 07")
ODR_Finish

[nltk_data] Downloading package wordnet to C:\Users\Diego
[nltk_data]     Ojeda\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\DIEGOO~1\AppData\Local\Temp/ipykernel_14212/1590325515.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_val["Target"] = le.inverse_transform(y_val)


'08/07/2011 22:00'

In [5]:
df,respuesta, ODR_Finish = identificacion_OCM("MOR BL 07")
#Rev = identificacion_OCM("MOR BL 07")

app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.Div(respuesta, style={'backgroundColor': 'red', 'color': 'white', 'width': '20%'}),
    dash_table.DataTable(
        data=df.to_dict('records'),
        style_header={
            'backgroundColor': '#de6e56',
            'fontWeight': 'bold'
        },
        columns=[{'id': c, 'name': c} for c in df.columns],
        style_data={
            'whiteSpace': 'normal',
            'height': 'auto',
            'lineHeight': '15px'
        },
        fixed_rows={'headers': True},
        style_cell={'textAlign': 'left','minWidth': 5, 'maxWidth': 300, 'width': 50}, # left align text in columns for readability
        style_cell_conditional=[
            {'if': {'column_id': 'Siglas'},'width': '3%','textAlign': 'center'},
            {'if': {'column_id': 'Target'},'width': '3%','textAlign': 'center'},
            {'if': {'column_id': 'Nombre'},'width': '5%','textAlign': 'center'},
            {'if': {'column_id': 'Desde'},'width': '10%','textAlign': 'center'},
            {'if': {'column_id': 'Hasta'},'width': '10%','textAlign': 'center'},
            {'if': {'column_id': 'MD From (ft)'},'width': '6%','textAlign': 'center'},
            {'if': {'column_id': 'MD to (ft)'},'width': '6%','textAlign': 'center'},
            {'if': {'column_id': 'Codigo'},'width': '6%','textAlign': 'center'},
            {'if': {'column_id': 'Subcodigo'},'width': '6%','textAlign': 'center'},
            {'if': {'row_index': 'odd'},'backgroundColor': 'rgb(220, 220, 220)',}
        ],
    #page_size=30,  # we have less data in this example, so setting to 20
    style_table={'height': '1200px', 'overflowY': 'auto'},
    
    style_data_conditional=[
        {
            'if': {
                'filter_query': '{Target} = "OCM"'
            },
            'backgroundColor': '#df8879',
            'color': 'white'
        }
    ]
    )
])
@callback(Output('tbl_out', 'children'), Input('tbl', 'active_cell'))
    
def update_graphs(active_cell):
    return str(active_cell) if active_cell else "Click the table"

if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


[nltk_data] Downloading package wordnet to C:\Users\Diego
[nltk_data]     Ojeda\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\DIEGOO~1\AppData\Local\Temp/ipykernel_14212/1590325515.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_val["Target"] = le.inverse_transform(y_val)
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [17/May/2022 17:41:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2022 17:41:13] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2022 17:41:13] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2022 17:41:13] "GET /_favicon.ico?v=2.3.1 HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2022 17:41:13] "GET /_dash-component-suites/dash/dash_t